In [ ]:
%env CUDA_LAUNCH_BLOCKING=1

In [ ]:
from experiments.attn.victory import *

In [ ]:
D = 32
B = 1024
T = 1000
device = 'cuda'

boards = load_boards().to(device)
worlds = Hex(boards.worlds)

n_boards, boardsize, _ = worlds.board.shape
head = VictoryHead(D)
model = common.FCModel(head, boardsize, D).to(device)

opt = torch.optim.Adam(model.parameters(), lr=1e-2)

with tqdm(total=T) as pbar:
    for t in range(T):
        idxs = torch.randint(0, n_boards, size=(B,), device=device)
        outputs = model(worlds[idxs].obs)
        targets = boards.target[idxs, 0].div(2).add(.5).int()

        loss = F.nll_loss(outputs, targets.long())

        opt.zero_grad()
        loss.backward()
        opt.step()

        pbar.update(1)
        pbar.set_description(f'{loss:.2f}')



In [ ]:
worlds[idxs].display(0)

In [ ]:
boards.target[idxs]